In [6]:
import torch.nn.functional as F
from torch.utils import data
import torch
import os
import random
import numpy as np

indice1=300
sample=64
cl=7

train = np.load("train.npy")
train = torch.from_numpy(np.stack(train, 0))
test = np.load("test.npy")
test = torch.from_numpy(np.stack(test, 0))
valid = np.load("valid.npy")
valid = torch.from_numpy(np.stack(valid, 0))

print(train.shape)
print(test.shape)
print(valid.shape)

label=np.arange(0,cl,1)
label = label.astype(np.float32)
train_label = torch.from_numpy(np.tile(label, indice1)) 
test_label  = torch.from_numpy(np.tile(label, 100))
valid_label = torch.from_numpy(np.tile(label, 2))

print(train_label.dtype)
print(train_label.shape)
print(test_label.shape)
print(valid_label.shape)


from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

train_dataset = TensorDataset(train, train_label)
valid_dataset = TensorDataset(valid, valid_label)
test_dataset = TensorDataset(test, test_label)
batch=cl*2

train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch,            
    shuffle= True,          
)

valid_loader = DataLoader(
    dataset = valid_dataset,
    batch_size = batch,           
    shuffle= True,          
)

test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = batch,     
    shuffle= True,            
)

torch.Size([2100, 64, 204])
torch.Size([700, 64, 204])
torch.Size([14, 64, 204])
torch.float32
torch.Size([2100])
torch.Size([700])
torch.Size([14])


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import argparse

from torch.autograd import Variable
import torch.optim as optim
from torch.utils import data
import os
import random


parser = argparse.ArgumentParser(description='Sequence Modeling')
parser.add_argument('--batch_size', type=int, default=cl*2, metavar='N',
                    help='batch size (default: 128)')
parser.add_argument('--cuda', action='store_true', default=False,
                    help='Disable cuda for training')
parser.add_argument('--dropout', type=float, default=0.45,
                    help='dropout applied to layers (default: 0.05)')
parser.add_argument('--clip', type=float, default=0.35,
                    help='gradient clip, -1 means no clip (default: -1)')
parser.add_argument('--epochs', type=int, default=30,
                    help='upper epoch limit (default: 20)')
parser.add_argument('--levels', type=int, default=4,
                    help='# of levels (default: 8)')
parser.add_argument('--log-interval', type=int, default=1, metavar='N',
                    help='report interval (default: 100')
parser.add_argument('--lr', type=float, default=1e-4,
                    help='initial learning rate (default: 2e-3)')
parser.add_argument('--optim', type=str, default='Adam',
                    help='optimizer to use (default: Adam)')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed (default: 1111)')
parser.add_argument('--seq_len', type=int, default=204)

args = parser.parse_args([])

if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")
print(args)

torch.manual_seed(args.seed)
batch_size = args.batch_size  
print(batch_size)
epochs = args.epochs 
input_channels = sample
print(input_channels)
channel_sizes = [input_channels*2] * args.levels   
n_classes = cl
print(n_classes)
steps = 0


import math
from einops import rearrange
from torch.nn import init
from torch.nn.utils import weight_norm

class Attention(nn.Module):

    def __init__(self, p=32, d_model=204, d_k=17*2, h=3, k=5, dropout=.1):

        super(Attention, self).__init__()
        self.fc_q = nn.Linear(d_model, h * d_k)
        self.fc_k = nn.Linear(d_model, h * d_k)
        self.fc_v = nn.Linear(d_model, h * d_k)
        self.fc_o = nn.Linear(d_model, h * d_k)

        self.softmax=nn.Softmax(-1)

        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_k
        self.h = h

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x, attention_mask=None, attention_weights=None):
        b, c, l = x.shape
        q = self.fc_q(x).view(b, c, self.h, self.d_k).permute(0, 2, 1, 3) 
        k = self.fc_k(x).view(b, c, self.h, self.d_k).permute(0, 2, 3, 1) 
        v = self.fc_v(x).view(b, c, self.h, self.d_v).permute(0, 2, 1, 3) 
        att = torch.matmul(q, k) / np.sqrt(self.d_k)  
        if attention_weights is not None:
            att = att * attention_weights

        att = torch.softmax(att, dim=1)
        att = torch.matmul(att, v).permute(0, 2, 1, 3).contiguous().view(b, c, self.h * self.d_v) # (b_s, nq, h*d_v)
        att = self.fc_o(att)
        att = att+x  
        return att

class Transformer(nn.Module):
    def __init__(self, input_size, output_size,p,l):
        super(Transformer, self).__init__()

        self.attention = Attention(p,l,l//3)

        self.length=l
        self.linear = nn.Linear(l*4, output_size)

        self.l1 = nn.Linear(204, l) 
        self.relu1=nn.ReLU()
        self.l2=nn.Linear(204, l)      
        self.ln = nn.LayerNorm([p,l],elementwise_affine = True)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        att = self.attention(x)
        out = self.relu1(self.l1(att))
        out = self.dropout(self.ln(self.l2(out)+att))

        y2 = F.adaptive_avg_pool2d(out.permute(0, 2, 1), [self.length,4])
        y3 = y2.view(y2.size(0), -1)
        o = self.linear(y3)

        o = F.log_softmax(o, dim=1).contiguous()

        return o

Namespace(batch_size=14, cuda=False, dropout=0.45, clip=0.35, epochs=30, levels=4, log_interval=1, lr=0.0001, optim='Adam', seed=1111, seq_len=204)
14
64
7


In [18]:
from torchsummary import summary

model = Transformer(input_channels, n_classes, (64//4)*4, 51*4)
if args.cuda:
    model.cuda()


summary(model, input_size=[(input_channels, 204)], batch_size=batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [14, 64, 204]          41,820
            Linear-2              [14, 64, 204]          41,820
            Linear-3              [14, 64, 204]          41,820
            Linear-4              [14, 64, 204]          41,820
     MUSEAttention-5              [14, 64, 204]               0
            Linear-6              [14, 64, 204]          41,820
              ReLU-7              [14, 64, 204]               0
            Linear-8              [14, 64, 204]          41,820
         LayerNorm-9              [14, 64, 204]          26,112
          Dropout-10              [14, 64, 204]               0
           Linear-11                    [14, 7]           5,719
Total params: 282,751
Trainable params: 282,751
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.70
Forward/

In [21]:
import torchmetrics
from sklearn.metrics import confusion_matrix

lr = args.lr
optimizer = getattr(optim, args.optim)(model.parameters(), lr=lr)

losses = []
loss_test = []
accuracy = []
fm = []
atts= []
acc_valid = []


def train(ep):
    train_loss = 0
    model.train()
    for batch_idx, (bands, target) in enumerate(train_loader):
 #       if args.cuda: bands, target = bands.cuda(), target.cuda()
        bands, target = Variable(bands), Variable(target)
        optimizer.zero_grad()
        output = model(bands)
        loss = F.nll_loss(output, target.long())
        loss.backward()
        if args.clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()
        train_loss += loss
        if batch_idx % args.log_interval == 0:
            los = train_loss.item()/args.log_interval
            losses.append(los)
            train_loss = 0

def test(loader):
    model.eval()
    test_acc = torchmetrics.Accuracy()
    test_fm = torchmetrics.ConfusionMatrix(threshold=1./16,num_classes=16)
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for bands, target in loader:
#            if args.cuda: bands, target = bands.cuda(), target.cuda()
            with torch.no_grad():
                bands, target = Variable(bands), Variable(target)
            pred = model(bands)
            test_loss += F.nll_loss(pred, target.long()).item()
            correct += (pred.argmax(1) == target.long()).type(torch.float).sum().item()
            test_acc(pred.argmax(1), target.long())
            test_fm(pred.argmax(1), target.long())

    test_loss /= len(test_loader)
    loss_test.append(test_loss)
    correct /= len(test_loader.dataset)

    total_acc = test_acc.compute()
    total_acc = torch.mean(total_acc)
    accuracy.append(total_acc)

    total_fm = test_fm.compute()
    fm.append(total_fm)

       
    print(f"Accuracy: {(100 * correct):>0.1f}%, "f"Avg loss: {test_loss:>8f}, "f"torch metrics acc: {total_acc}%")

    test_fm.reset()

    return test_loss

def valid(loader):
    model.eval()
    test_acc = torchmetrics.Accuracy()
    test_fm = torchmetrics.ConfusionMatrix(threshold=1./16,num_classes=16)
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for bands, target in loader:
#            if args.cuda: bands, target = bands.cuda(), target.cuda()
            with torch.no_grad():
                bands, target = Variable(bands), Variable(target)
            pred = model(bands)
            test_loss += F.nll_loss(pred, target.long()).item()
            correct += (pred.argmax(1) == target.long()).type(torch.float).sum().item()
            test_acc(pred.argmax(1), target.long())
            test_fm(pred.argmax(1), target.long())

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    total_acc = test_acc.compute()
    acc_valid.append(total_acc)

    total_fm = test_fm.compute()
    fm.append(total_fm)
       
    print(f"Valid Accuracy: {(100 * correct):>0.1f}%, "f"Avg loss: {test_loss:>8f}, "f"torch metrics acc: {total_acc}%")

    test_acc.reset()
    test_fm.reset()

    return test_loss

if __name__ == "__main__":

    for epoch in range(1, epochs+1):
        train(epoch)
        if epoch > 15:
            print("Epoch: ",epoch)
            print("TEST:") 
            test(test_loader)
#            print("VALID:")
#            valid(valid_loader)
            print("\n")
            
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr 

Epoch:  16
TEST:
Accuracy: 91.3%, Avg loss: 0.285476, torch metrics acc: 0.9128571152687073%


Epoch:  17
TEST:
Accuracy: 90.5%, Avg loss: 0.294173, torch metrics acc: 0.9049999713897705%


Epoch:  18
TEST:
Accuracy: 91.6%, Avg loss: 0.269252, torch metrics acc: 0.9164285659790039%


Epoch:  19
TEST:
Accuracy: 91.1%, Avg loss: 0.270171, torch metrics acc: 0.9114285707473755%


Epoch:  20
TEST:
Accuracy: 91.5%, Avg loss: 0.270339, torch metrics acc: 0.9150000214576721%


Epoch:  21
TEST:
Accuracy: 91.7%, Avg loss: 0.263640, torch metrics acc: 0.9171428680419922%


Epoch:  22
TEST:
Accuracy: 92.2%, Avg loss: 0.260296, torch metrics acc: 0.9221428632736206%


Epoch:  23
TEST:
Accuracy: 92.0%, Avg loss: 0.262711, torch metrics acc: 0.9200000166893005%


Epoch:  24
TEST:
Accuracy: 92.0%, Avg loss: 0.259574, torch metrics acc: 0.9200000166893005%


Epoch:  25
TEST:
Accuracy: 92.2%, Avg loss: 0.259404, torch metrics acc: 0.9221428632736206%


Epoch:  26
TEST:
Accuracy: 92.3%, Avg loss: 0.2597